In [1]:
day1_filepath = '../data/day_01_input.csv'
day2_filepath = '../data/day_02_input.csv'
day3_filepath = '../data/day_03_input.csv'
day4_filepath = '../data/day_04_input.csv'

# day1_filepath = '../data/day_01_input.csv'
# day1_filepath = '../data/day_01_input.csv'
# day1_filepath = '../data/day_01_input.csv'

In [2]:
# day 1
with open(day1_filepath) as file:
    num_increasing = 0
    # num_decreasing = 0
    step = 1
    previous_depth = int(file.readline())
    while True:
        try:
            current_depth = int(file.readline())
            step += 1
        except: 
            break
        diff = previous_depth - current_depth
        if diff < 0:
            num_increasing +=1
        # else:
        #     num_decreasing +=1
        previous_depth = current_depth

print('read in', step, 'depth measurements and found', num_increasing, 'of them increasing over previous step')

read in 2000 depth measurements and found 1602 of them increasing over previous step


Considering every single measurement isn't as useful as you expected: there's just too much noise in the data. Instead, consider sums of a three-measurement sliding window. Again considering the above example:

```
199  A
200  A B
208  A B C
210    B C D
200  E   C D
207  E F   D
240  E F G
269    F G H
260      G H
263        H
```
Start by comparing the first and second three-measurement windows. The measurements in the first window are marked `A (199, 200, 208)`; their sum is `199 + 200 + 208 = 607`. The second window is marked `B (200, 208, 210)`; its sum is 618. The sum of measurements in the second window is larger than the sum of the first, so this first comparison increased.

Your goal now is to count the number of times the sum of measurements in this sliding window increases from the previous sum. So, compare `A` with `B,` then compare `B` with `C`, then `C` with `D`, and so on. Stop when there aren't enough measurements left to create a new three-measurement sum.

In the above example, the sum of each three-measurement window is as follows:
```
A: 607 (N/A - no previous sum)
B: 618 (increased)
C: 618 (no change)
D: 617 (decreased)
E: 647 (increased)
F: 716 (increased)
G: 769 (increased)
H: 792 (increased)
```
In this example, there are 5 sums that are larger than the previous sum.
Consider sums of a three-measurement sliding window. How many sums are larger than the previous sum?

In [3]:
with open(day1_filepath) as file:
    num_increasing = 0
    # num_decreasing = 0
    previous_window = []
    for _ in range(3):
        previous_window.append(int(file.readline()))
    current_window = previous_window[-2:]
    next_window = [previous_window[-1]]
    num_steps = 3
    # while num_steps <10:
    while True:
        try:
            current_window.append(int(file.readline()))
        except:
            break
        num_steps +=1
        next_window.append(current_window[-1])
        # print(
        #     'A', previous_window, sum(previous_window),
        #     'B', current_window,  sum(current_window),
        #     'C', next_window, sum(next_window)
        # )
        diff = sum(previous_window) - sum(current_window)
        if diff < 0:
            num_increasing +=1
        previous_window = current_window
        current_window  = next_window
        next_window     = [current_window[-1]]

print('read in', num_steps, 'depth measurements and found', num_increasing, ' 3-measurement windows increasing over the previous')

read in 2000 depth measurements and found 1633  3-measurement windows increasing over the previous


In [4]:
# day 2 second version
window_size = 3
num_increasing = 0
running_window = []
with open(day1_filepath) as file:
    # initialise running window of 4 numbers
    for _ in range(window_size):
        running_window.append(int(file.readline()))
    step_num = window_size
    while True:
        try:
            running_window.append(int(file.readline()))
        except:
            break
        step_num +=1
        diff = sum(running_window[:window_size]) - sum(running_window[-window_size:])
        if diff < 0: # current>previous
            num_increasing +=1
        del running_window[0]

print('read in', step_num, 'depth measurements and found', 
    num_increasing, 'measurement windows of size', window_size, 
    'increasing depth over the previous window'
)

read in 2000 depth measurements and found 1633 measurement windows of size 3 increasing depth over the previous window


# --- Day 2: Dive! ---

Now, you need to figure out how to pilot this thing. It seems like the submarine can take a series of commands like forward 1, down 2, or up 3:

- `forward X` increases the horizontal position by X units.
- `down X` increases the depth by X units.
- `up X` decreases the depth by X units.

Note that since you're on a submarine, `down` and `up` affect your depth, and so they have the opposite result of what you might expect. The submarine seems to already have a planned course (your puzzle input). You should probably figure out where it's going. For example:

```
forward 5
down 5
forward 8
up 3
down 8
forward 2
```
Your horizontal position and depth both start at 0. The steps above would then modify them as follows:

- `forward 5` adds 5 to your horizontal position, a total of 5.
- `down 5` adds 5 to your depth, resulting in a value of 5.
- `forward 8` adds 8 to your horizontal position, a total of 13.
- `up 3` decreases your depth by 3, resulting in a value of 2.
- `down 8` adds 8 to your depth, resulting in a value of 10.
- `forward 2` adds 2 to your horizontal position, a total of 15.
After following these instructions, you would have a horizontal position of 15 and a depth of 10. (Multiplying these together produces 150.)

Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [5]:
x,d,step = 0,0,0
with open(day2_filepath) as file:
# with open('data/temp') as file:
    while True:
        try:
            direction, stepsize = file.readline().split(' ')
        except: 
            break
        step +=1
        stepsize = int(stepsize)
        if direction == 'forward':
            x += stepsize
        elif direction == 'down':
            d += stepsize
        elif direction == 'up':
            d -= stepsize

print('step ', step, ':', direction, 'by', stepsize, ':-----> (', x, ',', d,')')
print('multiplied:', x*d)

step  1000 : forward by 6 :-----> ( 2007 , 747 )
multiplied: 1499229


# --- Part Two ---

Based on your calculations, the planned course does not to make any sense. You find the submarine manual and discover that the process is actually slightly more complicated.

In addition to horizontal `position` and `depth`, you'll also need to track a third value, `aim`, which also starts at 0. The commands also mean something entirely different than you first thought:

- `down X` increases your aim by X units.
- `up X` decreases your aim by X units.
- `forward X` does two things:
    + It increases your horizontal position by X units.
    + It increases your depth by your aim multiplied by X.
Again note that since you're on a submarine, down and up do the opposite of what you might expect: "down" means aiming in the positive direction.
Now, the above example does something different:

- `forward 5` adds 5 to your horizontal position, a total of 5. Because your `aim` is 0, your `depth` does not change.
- `down 5` adds 5 to your `aim`, resulting in a value of 5.
- `forward 8` adds 8 to your horizontal position, a total of 13. Because your `aim` is 5, your `depth` increases by 8x5=40.
- `up 3` decreases your aim by 3, resulting in a value of 2.
- `down 8` adds 8 to your aim, resulting in a value of 10.
- `forward 2` adds 2 to your horizontal position, a total of 15. Because your aim is 10, your depth increases by 2x10=20 to a total of 60.

After following these new instructions, you would have a horizontal position of 15 and a depth of 60. (Multiplying these produces 900.)

Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [6]:
x,aim,depth,step = 0,0,0,0
with open(day2_filepath) as file:
# with open('data/temp') as file:
    while True:
        try:
            direction, stepsize = file.readline().split(' ')
        except: 
            break
        step +=1
        stepsize = int(stepsize)
        if direction == 'forward':
            x += stepsize
            depth += stepsize*aim
        elif direction == 'down':
            aim += stepsize
        elif direction == 'up':
            aim -= stepsize

print('step ', step, ':', direction, 'by', stepsize, ':-----> (', x, ',', depth,')')
print('multiplied:', x*depth)

step  1000 : forward by 6 :-----> ( 2007 , 668080 )
multiplied: 1340836560


# --- Day 3: Binary Diagnostic ---

The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.
The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the power consumption.
You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the gamma rate and the epsilon rate). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.
Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report. For example, given the following diagnostic report:
```
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
```
Considering only the first bit of each number, there are five 0 bits and seven 1 bits. Since the most common bit is 1, the first bit of the gamma rate is 1.
The most common second bit of the numbers in the diagnostic report is 0, so the second bit of the gamma rate is 0.
The most common value of the third, fourth, and fifth bits are 1, 1, and 0, respectively, and so the final three bits of the gamma rate are 110.
So, the gamma rate is the binary number `10110`, or 22 in decimal.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is `01001`, or 9 in decimal. Multiplying the gamma rate (22) by the epsilon rate (9) produces the power consumption, 198.

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. What is the power consumption of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [7]:
from operator import add
with open(day3_filepath) as file:
    line = 1
    first_line = file.readline().rstrip('\n')
    counts = [int(bit) for bit in first_line]
    # print(line, first_line, counts)
    # while line <19:
    while True:
        new_line = file.readline().rstrip('\n')
        if not new_line:
            break
        line +=1
        counts = list(map(add, counts, [int(bit) for bit in new_line]))
        # print(line, new_line, counts)
        
half = line/2
print(line, half, counts)
test_list = [1 if count == half else 0 for count in counts]
if sum(test_list) > 0:    
    print('warning! a draw is unexpected!', line, half, test_list, counts)
gamma_list = [1 if count > half  else 0 for count in counts]
gamma = int("".join(str(bit) for bit in gamma_list),2)
epsilon_list = [1 if count < half  else 0 for count in counts]
epsilon = int("".join(str(bit) for bit in epsilon_list),2)
nrg = gamma * epsilon

print('gamma_list', gamma_list)
print('epsilon_list', epsilon_list)
print('test_list', test_list)
print('gamma', gamma)
print('epsilon', epsilon)
print('energy', nrg)

1000 500.0 [497, 509, 520, 507, 491, 496, 511, 492, 505, 516, 478, 497]
gamma_list [0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0]
epsilon_list [1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1]
test_list [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
gamma 1836
epsilon 2259
energy 4147524


In [8]:
vec = [1,0,1,0,1,0]
flip = [1 if bit == 0 else 0 for bit in vec]
n= int("".join(str(bit) for bit in vec),2) * int("".join(str(bit) for bit in flip),2)
print(n)
b= []
while(n>0):
    d=n%2
    b.append(d)
    n=n//2
b.reverse()
print("multiply", vec, 'and', flip, 'to get')
for i in b:
    print(i,end="")
    
def fn(s, base=10):
    prefix = s[0:2]
    if prefix == '0x':
        base = 16
    elif prefix == '0b':
        base = 2
    return bin(int(s, base))


print(fn('15'))
print(fn('0xF'))
print(fn('0b1111'))


882
multiply [1, 0, 1, 0, 1, 0] and [0, 1, 0, 1, 0, 1] to get
11011100100b1111
0b1111
0b1111


# --- Part Two ---
Next, you should verify the life support rating, which can be determined by multiplying the _oxygen generator rating_ by the _CO2 scrubber_ rating.

Both the _oxygen generator rating_ and the _CO2 scrubber rating_ are values that can be found in your diagnostic report - finding them is the tricky part. Both values are located using a similar process that involves filtering out values until only one remains. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and consider just the first bit of those numbers. Then:

- Keep only numbers selected by the _bit criteria_ for the type of rating value for which you are searching. Discard numbers which do not match the bit criteria.
- If you only have one number left, stop; this is the rating value for which you are searching.
- Otherwise, repeat the process, considering the next bit to the right.

The _bit criteria_ depends on which type of rating value you want to find:
- To find _oxygen generator rating_, determine the most common value (`0` or `1`) in the current bit position, and keep only numbers with that bit in that position. If `0` and `1` are equally common, keep values with a `1` in the position being considered.
- To find _CO2 scrubber rating_, determine the least common value (`0` or `1`) in the current bit position, and keep only numbers with that bit in that position. If `0` and `1` are equally common, keep values with a `0` in the position being considered.

For example, to determine the _oxygen generator rating_ value using the same example diagnostic report from above:
- Start with all 12 numbers and consider only the first bit of each number. There are more `1` bits (7) than `0` bits (5), so keep only the 7 numbers with a `1` in the first position: `11110`, `10110`, `10111`, `10101`, `11100`, `10000`, and `11001`.
- Then, consider the second bit of the 7 remaining numbers: there are more `0` bits (4) than `1` bits (3), so keep only the 4 numbers with a `0` in the second position: `10110`, `10111`, `10101`, and `10000`.
- In the third position, three of the four numbers have a `1`, so keep those three: `10110`, `10111`, and `10101`.
- In the fourth position, two of the three numbers have a `1`, so keep those two: `10110` and `10111`.
- In the fifth position, there are an equal number of `0` bits and `1` bits (one each). So, to find the oxygen generator rating, keep the number with a `1` in that position: `10111`.
- As there is only one number left, stop; the _oxygen generator rating_ is `10111`, or 23 in decimal.

Then, to determine the CO2 scrubber rating value from the same example above:

- Start again with all 12 numbers and consider only the first bit of each number. There are fewer `0` bits (5) than `1` bits (7), so keep only the 5 numbers with a 0 in the first position: `00100`, `01111`, `00111`, `00010`, and `01010`.
- Then, consider the second bit of the 5 remaining numbers: there are fewer `1` bits (2) than `0` bits (3), so keep only the 2 numbers with a `1` in the second position: `01111` and `01010`.
- In the third position, there are an equal number of `0` bits and `1` bits (one each). So, to find the CO2 scrubber rating, keep the number with a `0` in that position: `01010`.
- As there is only one number left, stop; the _CO2 scrubber rating_ is `01010`, or 10 in decimal.
- Finally, to find the life support rating, multiply the _oxygen generator rating_ (23) by the _CO2 scrubber rating_ (10) to get 230.

Use the binary numbers in your diagnostic report to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together. What is the life support rating of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [9]:
# with open('../data/temp') as file:
with open(day3_filepath) as file:
    # read in once and store in memory
    candidates = [bit_str.rstrip('\n') for bit_str in file.readlines()]
# could use counts from previous calculation, or just recompute it
len_num = len(candidates[0]) # assume uniform data. length of first is length of all

def get_rating(candidates, system='oxygen'):
    num_num = len(candidates) # number of binary numbers to search among
    rating = None
    for digit in range(len_num):
        num_ones = 0
        for number in range(num_num):
            num_ones += int(candidates[number][digit])
        if num_ones >= num_num/2:
            # 1 is the majority digit, or they are even
            oxygen_filter = '1'
            co2_filter = '0'
        else:
            # 0 is the majority digit
            oxygen_filter = '0'
            co2_filter = '1'
        if system == 'oxygen':
            candidates = [number for number in candidates if number[digit] == oxygen_filter]
        elif system == 'co2':
            candidates = [number for number in candidates if number[digit] == co2_filter]
        else:
            print('unsupported system! only oxygen or co2 are supported, you entered:', system)
        num_num = len(candidates) # number of candidates left
        if num_num == 1:
            # yay! we're done!
            rating = candidates[0]
            break

    if num_num != 1:
        print('fuck. how did this happen?', candidates, num_num)
    return rating

oxygen_rating = get_rating(candidates, 'oxygen')
oxygen_rating_int = int("".join(bit for bit in oxygen_rating),2)
co2_rating    = get_rating(candidates, 'co2')
co2_rating_int = int("".join(bit for bit in co2_rating),2)
life_support_rating = oxygen_rating_int*co2_rating_int
print('oxygen system rating is', oxygen_rating, '=', oxygen_rating_int)
print('co2    system rating is',    co2_rating, '=',    co2_rating_int)
print('life  support rating is',    life_support_rating)

oxygen system rating is 010110010011 = 1427
co2    system rating is 100111000110 = 2502
life  support rating is 3570354


# --- Day 4: Giant Squid ---

You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

```
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
```

After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):



In [10]:
# define required functions

def update_match(board, match, draw, board_size):
    for row in range(board_size):
        for column in range(board_size):
            if board[row][column] == draw:
                match[row][column] = True
                return match
    return match


def check_for_winners(match, board_size):
    # check if any row is complete
    # check if any column is complete:
    for column in range(board_size):
        if sum(match[:,column]) == board_size:
            return True, 'column', column

    for row in range(board_size):
        if sum(match[row,:]) == board_size:
            return True, 'row', row

    return False, None, -1


def compute_score(winner_board, winner_match, latest_draw, board_size):
    """
        The score of the winning board can now be calculated. 
        Start by finding the sum of all unmarked numbers on that board; 
        in this case, the sum is 188. 
        Then, multiply that sum by the number that was just called when 
        the board won, 24, to get the final score, 188 * 24 = 4512
    """
    score = 0
    for row in range(board_size):
        for column in range(board_size):
            if not winner_match[row][column]:
                score += winner_board[row][column]
    print('score sum = ', score, 'score multplier = ', latest_draw)
    score *= latest_draw
    print('final score = ', score)
    
    return score

In [11]:
# set up the game from input file
import numpy as np
board_size = 5
boards = []
# with open('../data/temp.txt', 'r') as file:
with open(day4_filepath, 'r') as file:
    # first line contains draws
    draws = [int(number) for number in file.readline().rstrip('\n').split(',')]
    while line := file.readline():
        line = line.rstrip('\n')
        if line == '':
            # latest line read was a blank line that precedes a new board description
            board = np.zeros((board_size, board_size), dtype=np.int8)
            # populate new board with the next 5 rows:
            for row in range(board_size):
                board[row] = [ int(number) for number in file.readline().rstrip('\n').split() ]
            # print(board)
            boards.append(board)
        else: 
            print('out of sync?', line)
            
num_boards = len(boards)
matches = [
    np.full((board_size, board_size), fill_value=False, dtype=bool)
    for _ in range(num_boards)
]
print('-------------------------')
print ('there were', num_boards, 'boards and', len(draws), 'draws in input file.')
print('-------------------------')
# matches
# step through the random draws. update the game status.
winner_found = False
for draw_id in range(len(draws)):
    latest_draw = draws[draw_id]
    # update found matches in all boards
    for board_id in range(num_boards):
        # print(matches[board_id], len(matches[board_id]))
        temp =\
            update_match(boards[board_id],
                         matches[board_id], 
                         latest_draw, 
                         board_size)
        matches[board_id] = temp
    if draw_id >= 5:
        # search for winners
        for board_id in range(num_boards):
            winner_found, dimension, index =\
                check_for_winners(matches[board_id],
                                  board_size)
            if winner_found:
                print('-------------------------')
                print('after', draw_id+1, 'rounds, board number', board_id+1, 'is a winner!')
                print(boards[board_id])
                print(matches[board_id])
                print(dimension, index, 'is completed by', draws[:draw_id+1])
                print('-------------------------')
                compute_score(boards[board_id], matches[board_id], latest_draw, board_size)
                break
    if winner_found:
        break

-------------------------
there were 100 boards and 100 draws in input file.
-------------------------
-------------------------
after 27 rounds, board number 61 is a winner!
[[19 85 36 73 71]
 [65 62 14 52  3]
 [30 83 44 41  5]
 [55 15  0 61 95]
 [28 13 32 31 88]]
[[False False False False False]
 [False False False False False]
 [False False  True False False]
 [ True False False False False]
 [ True  True  True  True  True]]
row 4 is completed by [46, 79, 77, 45, 57, 34, 44, 13, 32, 88, 86, 82, 91, 97, 89, 1, 48, 31, 18, 10, 55, 74, 24, 11, 80, 78, 28]
-------------------------
score sum =  810 score multplier =  28
final score =  22680


# --- Part Two ---

On the other hand, it might be wise to try a different strategy: let the giant squid win.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to figure out which board will win last and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after 13 is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to 148 for a final _score_ of `148 * 13 = 1924`.

Figure out which board will win last. Once it wins, what would its final score be?

In [12]:
# set up the game from input file
import numpy as np
board_size = 5
boards = []
# with open('../data/temp.txt', 'r') as file:
with open(day4_filepath, 'r') as file:
    # first line contains draws
    draws = [int(number) for number in file.readline().rstrip('\n').split(',')]
    while line := file.readline():
        line = line.rstrip('\n')
        if line == '':
            # latest line read was a blank line that precedes a new board description
            board = np.zeros((board_size, board_size), dtype=np.int8)
            # populate new board with the next 5 rows:
            for row in range(board_size):
                board[row] = [ int(number) for number in file.readline().rstrip('\n').split() ]
            # print(board)
            boards.append(board)
        else: 
            print('out of sync?', line)
            
num_boards = len(boards)
matches = [
    np.full((board_size, board_size), fill_value=False, dtype=bool)
    for _ in range(num_boards)
]
print('--------------------------------------------------')
print ('there were', num_boards, 'boards and', len(draws), 'draws in input file.')
print('--------------------------------------------------')
# matches
# step through the random draws. update the game status.
winner_found = False
last_winner_found = False
winner_ids = set()
for draw_id in range(len(draws)):
    latest_draw = draws[draw_id]
    # update found matches in all boards
    for board_id in range(num_boards):
        # print(matches[board_id], len(matches[board_id]))
        matches[board_id] =\
            update_match(boards[board_id],
                         matches[board_id], 
                         latest_draw, 
                         board_size)
    # if draw_id >= 5:
        # search for winners
    for board_id in range(num_boards):
        winner_found, dimension, index =\
            check_for_winners(matches[board_id],
                              board_size)
        if winner_found:
            if board_id not in winner_ids:
                print('after', draw_id+1, 'rounds, board number', board_id+1, 'is a winner!')
                winner_ids.add(board_id) 
                if len(winner_ids) == num_boards:
                    last_winner_found = True
                    # the last board to complete a column or a row
                    print('--------------------------------------------------')
                    print('after', draw_id+1, 'rounds, board number', board_id+1, 'is the last winner!')
                    print(boards[board_id])
                    print(matches[board_id])
                    print(dimension, index, 'is completed by', draws[:draw_id+1])
                    if dimension == 'row':
                        print(matches[board_id][index][:])
                        print(boards[board_id][index][:])
                    elif dimension == 'column':
                        print(matches[board_id][:][index])
                        print(boards[board_id][:][index])
                    print('--------------------------------------------------')
                    compute_score(boards[board_id], matches[board_id], latest_draw, board_size)
                    break
    if last_winner_found:
        break

--------------------------------------------------
there were 100 boards and 100 draws in input file.
--------------------------------------------------
after 27 rounds, board number 61 is a winner!
after 28 rounds, board number 52 is a winner!
after 29 rounds, board number 69 is a winner!
after 31 rounds, board number 46 is a winner!
after 32 rounds, board number 72 is a winner!
after 34 rounds, board number 75 is a winner!
after 35 rounds, board number 32 is a winner!
after 35 rounds, board number 80 is a winner!
after 38 rounds, board number 3 is a winner!
after 39 rounds, board number 95 is a winner!
after 41 rounds, board number 38 is a winner!
after 42 rounds, board number 31 is a winner!
after 44 rounds, board number 71 is a winner!
after 44 rounds, board number 88 is a winner!
after 44 rounds, board number 92 is a winner!
after 45 rounds, board number 17 is a winner!
after 45 rounds, board number 18 is a winner!
after 46 rounds, board number 19 is a winner!
after 47 rounds, boa

# --- Day 5: Hydrothermal Venture ---

You come across a field of hydrothermal vents on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible. They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:

```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```
Each line of vents is given as a line segment in the format `x1,y1 -> x2,y2` where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

An entry like `1,1 -> 1,3` covers points `1,1`, `1,2`, and `1,3`.
An entry like `9,7 -> 7,7` covers points `9,7`, `8,7`, and `7,7`.
For now, only consider horizontal and vertical lines: lines where either `x1 = x2` or `y1 = y2`.

So, the horizontal and vertical lines from the above list would produce the following diagram:
```
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
```
In this diagram, the top left corner is `0,0` and the bottom right corner is `9,9`. Each position is shown as the number of lines which cover that point or `.` if no line covers that point. The top-left pair of `1`s, for example, comes from `2,2 -> 2,1`; the very bottom row is formed by the overlapping lines `0,9 -> 5,9` and `0,9 -> 2,9`.

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a `2` or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

In [98]:
def update_orthogonal_track(tracks, start_x, start_y, stop_x, stop_y):
    # adds new track to set of tracked points. reports repeatedly trodden points.
    # works only for tracks where either 
    #  1. start_x = stop_x or 2. start_y = stop_y
    repeated_points = set({})
    for x in range(min([start_x, stop_x]), max(start_x, stop_x)+1):
        for y in range(min([start_y, stop_y]), max(start_y, stop_y)+1):
            if (x,y) in tracks:
                # found a new repeated point
                repeated_points.add((x,y))
            else:
                # new point, add to tracks
                tracks.add((x,y))
    return tracks, repeated_points


def update_diagonal_track(tracks, start_x, start_y, stop_x, stop_y):
    # adds new track to set of tracked points. reports repeated points.
    # works only for diagonal tracks, where:
    #   stop_x - start_x  = stop_y - start_y     
    repeated_points = set({})
    num_points = abs(stop_x - start_x) # range spanned by x
    if abs(stop_y - start_y) != num_points:
        print('wrong assumptions? deltas:', num_points, abs(stop_y - start_y))
    if start_y > stop_y: 
        # y decreases
        y_sign = -1
    else:
        y_sign = 1
    if start_x > stop_x:
        # x decreases
        x_sign = -1
    else:
        x_sign = 1
    for step in range(0, num_points+1):
        x = start_x + step*x_sign
        y = start_y + step*y_sign
        if (x,y) in tracks:
            # found a new repeated point
            repeated_points.add((x,y))
        else:
            # new point, add to tracks
            tracks.add((x,y))
    if (x,y) != (stop_x, stop_y):
        print('wrong assumptions?', (start_x, start_y), 'num points=', temp, (x,y), (stop_x, stop_y), x_sign, y_sign)
    return tracks, repeated_points

In [99]:
# first read in the oddly formatted data
already_covered_points = set({}) # set of points known to have at least one vent
multiple_crossings = set({}) # set of points known to  have more than one vent
max_x, min_x, max_y, min_y = 0, 0, 0, 0
num_lines = 0
with open('../data/day_05_input.csv', 'r') as file:
    while line := file.readline():
    # while num_lines < 100:
    #     line = file.readline()
        num_lines += 1
        # parse line input
        coord_str = [pair.split(',') for pair in line.rstrip('\n').split(' -> ')]
        [start_x, start_y, stop_x, stop_y] = [
            int(coord_str[0][0]),int(coord_str[0][1]),int(coord_str[1][0]), int(coord_str[1][1])
        ]
        max_x = max([max_x, start_x, stop_x])
        max_y = max([max_y, start_y, stop_y])
        
        if start_x==stop_x or start_y==stop_y:
            # only consider vertical or horizontal lines
            # print(num_lines, start_x, start_y, stop_x, stop_y, ':', multiple_crossings)
            already_covered_points, new_crossings = \
                update_orthogonal_track(already_covered_points, start_x, start_y, stop_x, stop_y)
        else:
            already_covered_points, new_crossings = \
                update_diagonal_track(already_covered_points, start_x, start_y, stop_x, stop_y)
        for new_crossing in new_crossings:
            # print('add new repeated point', new_crossing)
            multiple_crossings.add(new_crossing)

print(num_lines, max_x, max_y, len(multiple_crossings))

500 990 990 17741


--- Day 6: Lanternfish ---

The sea floor is getting steeper. Maybe the sleigh keys got carried this way?

A massive school of glowing lanternfish swims past. They must spawn quickly to reach such large numbers - maybe exponentially quickly? You should model their growth rate to be sure.

Although you know nothing about this specific species of lanternfish, you make some guesses about their attributes. Surely, each lanternfish creates a new lanternfish once every 7 days.

However, this process isn't necessarily synchronized between every lanternfish - one lanternfish might have 2 days left until it creates another lanternfish, while another might have 4. So, you can model each fish as a single number that represents the number of days until it creates a new lanternfish.

Furthermore, you reason, a new lanternfish would surely need slightly longer before it's capable of producing more lanternfish: two more days for its first cycle.

So, suppose you have a lanternfish with an internal timer value of 3:

After one day, its internal timer would become 2.
After another day, its internal timer would become 1.
After another day, its internal timer would become 0.
After another day, its internal timer would reset to 6, and it would create a new lanternfish with an internal timer of 8.
After another day, the first lanternfish would have an internal timer of 5, and the second lanternfish would have an internal timer of 7.
A lanternfish that creates a new fish resets its timer to 6, not 7 (because 0 is included as a valid timer value). The new lanternfish starts with an internal timer of 8 and does not start counting down until the next day.

Realizing what you're trying to do, the submarine automatically produces a list of the ages of several hundred nearby lanternfish (your puzzle input). For example, suppose you were given the following list:

3,4,3,1,2
This list means that the first fish has an internal timer of 3, the second fish has an internal timer of 4, and so on until the fifth fish, which has an internal timer of 2. Simulating these fish over several days would proceed as follows:
```
Initial state: 3,4,3,1,2
After  1 day:  2,3,2,0,1
After  2 days: 1,2,1,6,0,8
After  3 days: 0,1,0,5,6,7,8
After  4 days: 6,0,6,4,5,6,7,8,8
After  5 days: 5,6,5,3,4,5,6,7,7,8
After  6 days: 4,5,4,2,3,4,5,6,6,7
After  7 days: 3,4,3,1,2,3,4,5,5,6
After  8 days: 2,3,2,0,1,2,3,4,4,5
After  9 days: 1,2,1,6,0,1,2,3,3,4,8
After 10 days: 0,1,0,5,6,0,1,2,2,3,7,8
After 11 days: 6,0,6,4,5,6,0,1,1,2,6,7,8,8,8
After 12 days: 5,6,5,3,4,5,6,0,0,1,5,6,7,7,7,8,8
After 13 days: 4,5,4,2,3,4,5,6,6,0,4,5,6,6,6,7,7,8,8
After 14 days: 3,4,3,1,2,3,4,5,5,6,3,4,5,5,5,6,6,7,7,8
After 15 days: 2,3,2,0,1,2,3,4,4,5,2,3,4,4,4,5,5,6,6,7
After 16 days: 1,2,1,6,0,1,2,3,3,4,1,2,3,3,3,4,4,5,5,6,8
After 17 days: 0,1,0,5,6,0,1,2,2,3,0,1,2,2,2,3,3,4,4,5,7,8
After 18 days: 6,0,6,4,5,6,0,1,1,2,6,0,1,1,1,2,2,3,3,4,6,7,8,8,8,8
Each day, a 0 becomes a 6 and adds a new 8 to the end of the list, while each other number decreases by 1 if it was present at the start of the day.

In this example, after 18 days, there are a total of 26 fish. After 80 days, there would be a total of 5934.

Find a way to simulate lanternfish. How many lanternfish would there be after 80 days?

# --- Day 6: Lanternfish ---
The sea floor is getting steeper. Maybe the sleigh keys got carried this way? A massive school of glowing lanternfish swims past. They must spawn quickly to reach such large numbers - maybe exponentially quickly? You should model their growth rate to be sure. Although you know nothing about this specific species of lanternfish, you make some guesses about their attributes. Surely, each lanternfish creates a new lanternfish once every 7 days. However, this process isn't necessarily synchronized between every lanternfish - one lanternfish might have 2 days left until it creates another lanternfish, while another might have 4. So, you can model each fish as a single number that represents the number of days until it creates a new lanternfish. Furthermore, you reason, a new lanternfish would surely need slightly longer before it's capable of producing more lanternfish: two more days for its first cycle.

So, suppose you have a lanternfish with an internal timer value of 3:

- After one day, its internal timer would become 2.
- After another day, its internal timer would become 1.
- After another day, its internal timer would become 0.
- After another day, its internal timer would reset to 6, and it would create a new lanternfish with an internal timer of 8.
- After another day, the first lanternfish would have an internal timer of 5, and the second lanternfish would have an internal timer of 7.

A lanternfish that creates a new fish resets its timer to 6, not 7 (because 0 is included as a valid timer value). The new lanternfish starts with an internal timer of 8 and does not start counting down until the next day. Realizing what you're trying to do, the submarine automatically produces a list of the ages of several hundred nearby lanternfish (your puzzle input). For example, suppose you were given the following list:
```
3,4,3,1,2
```
This list means that the first fish has an internal timer of 3, the second fish has an internal timer of 4, and so on until the fifth fish, which has an internal timer of 2. Simulating these fish over several days would proceed as follows:
```
Initial state: 3,4,3,1,2
After  1 day:  2,3,2,0,1
After  2 days: 1,2,1,6,0,8
After  3 days: 0,1,0,5,6,7,8
After  4 days: 6,0,6,4,5,6,7,8,8
After  5 days: 5,6,5,3,4,5,6,7,7,8
After  6 days: 4,5,4,2,3,4,5,6,6,7
After  7 days: 3,4,3,1,2,3,4,5,5,6
After  8 days: 2,3,2,0,1,2,3,4,4,5
After  9 days: 1,2,1,6,0,1,2,3,3,4,8
After 10 days: 0,1,0,5,6,0,1,2,2,3,7,8
After 11 days: 6,0,6,4,5,6,0,1,1,2,6,7,8,8,8
After 12 days: 5,6,5,3,4,5,6,0,0,1,5,6,7,7,7,8,8
After 13 days: 4,5,4,2,3,4,5,6,6,0,4,5,6,6,6,7,7,8,8
After 14 days: 3,4,3,1,2,3,4,5,5,6,3,4,5,5,5,6,6,7,7,8
After 15 days: 2,3,2,0,1,2,3,4,4,5,2,3,4,4,4,5,5,6,6,7
After 16 days: 1,2,1,6,0,1,2,3,3,4,1,2,3,3,3,4,4,5,5,6,8
After 17 days: 0,1,0,5,6,0,1,2,2,3,0,1,2,2,2,3,3,4,4,5,7,8
After 18 days: 6,0,6,4,5,6,0,1,1,2,6,0,1,1,1,2,2,3,3,4,6,7,8,8,8,8
```
Each day, a 0 becomes a 6 and adds a new 8 to the end of the list, while each other number decreases by 1 if it was present at the start of the day.

In this example, after 18 days, there are a total of 26 fish. After 80 days, there would be a total of 5934.

Find a way to simulate lanternfish. How many lanternfish would there be after 80 days?

In [115]:
num_days = 80
with open('../data/day_06_input.csv', 'r') as file:
    fish_ages = [int(age_str) for age_str in file.readline().rstrip('\n').split(',')]

num_fish = len(fish_ages)
print('initially there are', num_fish, 'fish with aged from', min(fish_ages), 'to', max(fish_ages))
# fish_ages = [1,2,3,4,5]
for day in range(num_days):
    # print(day, fish_ages)
    for fish in range(num_fish):
        if fish_ages[fish] == 0:
            fish_ages.append(8)
            fish_ages[fish] = 6
            num_fish += 1
        else: 
            fish_ages[fish] -= 1

print('after', day+1, 'days, the number of fish is', len(fish_ages), num_fish)

initially there are 300 fish with aged from 1 to 5
after 80 days, the number of fish is 390923 390923


In [126]:
num_days = 256
# with open('../data/day_06_input.csv', 'r') as file:
#     fish_ages = [int(age_str) for age_str in file.readline().rstrip('\n').split(',')]
# compute how many offspring a single newspawn will produce in 256 days
fish_ages = [8] 
growth_rates = {0:1} 
num_fish = len(fish_ages)
print('initially there are', num_fish, 'fish with aged from', min(fish_ages), 'to', max(fish_ages))
# fish_ages = [1,2,3,4,5]
for day in range(num_days):
    print(day, len(fish_ages))
    for fish in range(num_fish):
        if fish_ages[fish] == 0:
            fish_ages.append(8)
            fish_ages[fish] = 6
            num_fish += 1
        else: 
            fish_ages[fish] -= 1
    growth_rates[day+1] = num_fish

print('after', day+1, 'days, the number of fish is', len(fish_ages), num_fish)

initially there are 1 fish with aged from 8 to 8
0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 2
10 2
11 2
12 2
13 2
14 2
15 2
16 3
17 3
18 4
19 4
20 4
21 4
22 4
23 5
24 5
25 7
26 7
27 8
28 8
29 8
30 9
31 9
32 12
33 12
34 15
35 15
36 16
37 17
38 17
39 21
40 21
41 27
42 27
43 31
44 32
45 33
46 38
47 38
48 48
49 48
50 58
51 59
52 64
53 70
54 71
55 86
56 86
57 106
58 107
59 122
60 129
61 135
62 156
63 157
64 192
65 193
66 228
67 236
68 257
69 285
70 292
71 348
72 350
73 420
74 429
75 485
76 521
77 549
78 633
79 642
80 768
81 779
82 905
83 950
84 1034
85 1154
86 1191
87 1401
88 1421
89 1673
90 1729
91 1939
92 2104
93 2225
94 2555
95 2612
96 3074
97 3150
98 3612
99 3833
100 4164
101 4659
102 4837
103 5629
104 5762
105 6686
106 6983
107 7776
108 8492
109 9001
110 10288
111 10599
112 12315
113 12745
114 14462
115 15475
116 16777
117 18780
118 19600
119 22603
120 23344
121 26777
122 28220
123 31239
124 34255
125 36377
126 41383
127 42944
128 49380
129 51564
130 58016
131 62475
132 67616
133 75638
134 

In [131]:
# import pickle
# output = open('../data/day6_temp.pkl', 'wb')
# pickle.dump(growth_rates, output)
# output.close()

# # read python dict back from the file
# pkl_file = open('../data/day6_temp.pkl', 'rb')
# growth_rates = pickle.load(pkl_file)
# pkl_file.close()


so, starting from a single new spawn (age 8), the number of fish in 256 days will be 3,369,186,778

only consider the original 300 fish. every time one of them spawns on day d, there are 256-d days left. a new fish spawned on day d has 256-d to generate their own branch of the family tree. look up that number, and add it to a count of fish. 

number of spawnings per age = 1 fish in 256 daysgrowth_rates after  1 day:  0 (age 0)
- after  2 days: 1 (age 6)  -> new fish (age 8) spawned who will, in turn spawn for 254 days
- after  3 days: 1 (age 5)
- after  4 days: 1 (age 4)
- after  5 days: 1 (age 3)
- after  6 days: 1 (age 2)
- after  7 days: 1 (age 1)
- after  8 days: 1 (age 0) 
- after  9 days: 2 (age 6)  -> new fish (age 8) spawned who will, in turn spawn for 247 days
...
- after 16 days: 3 (age 6) -> new fish spawned who will, in turn spawn...
...
- after 23 days: 4 (age 6) -> new fish spawned who will, in turn spawn...
...
- after n*7-5 days: n (age 6) -> new fish spawned who will, in turn spawn...
...
- after 254 days: 37 (age 6) -> new fish spawned who will, not have time to spawn.

wheras a fish of age 5 will spawn 
- after  1 day:   0 (age 4) 
- after  2 days:  0 (age 3) 
- after  3 days:  0 (age 2) 
- after  4 days:  0 (age 1) 
- after  5 days:  0 (age 0) 
- after  6 days:  1 (age 6) -> new fish (age 8) spawned who will, in turn spawn for 247 days
- after  7 days:  1 (age 5) 
- after  8 days:  1 (age 4) 
- after  9 days:  1 (age 3) 
- after 10 days:  1 (age 2) 
- after 11 days:  1 (age 1) 
- after 12 days:  1 (age 0) 
- after 13 days:  2 (age 6) -> new fish (age 8) spawned who will, in turn spawn for 247 days
...
- after n*7-1 days: n (age 6) -> new fish (age 8) spawned who will, in turn spawn for 247 days

wheras a fish of age 8 will spawn 
- after  1 day:   0 (age 7) 
- after  2 days:  0 (age 6) 
- after  3 days:  0 (age 5) 
- after  4 days:  0 (age 4) 
- after  5 days:  0 (age 3) 
- after  6 days:  0 (age 2) 
- after  7 days:  0 (age 1) 
- after  8 days:  1 (age 0) -> new fish (age 8) spawned who will, in turn spawn for 247 days
- after  9 days:  1 (age 6) 
- after 10 days:  1 (age 5) 
- after 11 days:  1 (age 4) 
- after 12 days:  1 (age 3) 
- after 12 days:  1 (age 2) 
- after 13 days:  1 (age 1)
- after 14 days:  2 (age 0) -> new fish (age 8) spawned who will, in turn spawn for 247 days
...
- after n*7-1 days: n (age 6) -> new fish (age 8) spawned who will, in turn spawn for 247 days

fish of starting age x, will spawn their nth spawn on day number n*7+1.


In [129]:
print(growth_rates[128]/growth_rates[64])
print(growth_rates[100]/growth_rates[50])
print('----------------')
print(growth_rates[256]/growth_rates[255])
print(growth_rates[201]/growth_rates[200])
print(growth_rates[100]/growth_rates[99])
print(growth_rates[99]/growth_rates[98])

257.1875
71.79310344827586
----------------
1.1034930561932377
1.1097744378175203
1.0863553352465432
1.061184939091916


In [139]:
num_days = 256
with open('../data/day_06_input.csv', 'r') as file:
    fish_ages = [int(age_str) for age_str in file.readline().rstrip('\n').split(',')]
# compute how many offspring a single newspawn will produce in 256 days
num_fish = len(fish_ages)
total_fish = num_fish 
print('initially there are', total_fish, 'fish with ages from', min(fish_ages), 'to', max(fish_ages))
for days_left in range(256,0,-1):
    for fish in range(num_fish):
        if fish_ages[fish] == 0:
            # one of the originals now spawns a new fish.
            # new fish + descendents will add to the tally 
            # a number, depending on how many days are left:
            total_fish += growth_rates[days_left]
            fish_ages[fish] = 6
        else: 
            fish_ages[fish] -= 1
    # print(days_left, total_fish)

print('after 256 days, the number of fish has grown to', total_fish)

initially there are 300 fish with ages from 1 to 5
after 256 days, the number of fish has grown to 1896062838926


10
9
8
7
6
5
4
3
2
1
0


answer lies between 1749945484914 (too low)
and 1896062838926 (too high)